In [1]:
from osgeo import gdal
import numpy as np
from shapely.geometry import Point, Polygon
from scipy.spatial import ConvexHull

data_path = '0_data/'

obs_clusters = gdal.Open(data_path + 'obs_clusters.tif')
obs_clusters_band = obs_clusters.GetRasterBand(1)
obs_clusters_data = obs_clusters.ReadAsArray()
row_length = obs_clusters.RasterYSize
col_length = obs_clusters.RasterXSize
tif_geotrans = obs_clusters.GetGeoTransform()
tif_proj = obs_clusters.GetProjection()

cluster_max = int(obs_clusters_data.max())
clusters_rowcols = []
clusters_vertices = []
clusters_vertices_poly = []
for i in range(cluster_max+1):
    clusters_rowcols.append([])
    clusters_vertices.append([])
    clusters_vertices_poly.append([])

In [2]:
for row in range(row_length):
    for col in range(col_length):
        cluster_id = obs_clusters_data[row, col]
        if cluster_id > 0:
            clusters_rowcols[cluster_id].append([row, col])


for cluster_i in range(len(clusters_rowcols)):
    if cluster_i > 0:
        cluster_i_points = np.array(clusters_rowcols[cluster_i])
        cluster_i_hull = ConvexHull(cluster_i_points)
        for vertex in cluster_i_hull.vertices:
            vertex_row = cluster_i_points[vertex, 0]
            vertex_col = cluster_i_points[vertex, 1]
            clusters_vertices[cluster_i].append([vertex_row, vertex_col]) 
            clusters_vertices_poly[cluster_i].append((vertex_row, vertex_col))  # tuple for following polygon create

In [3]:
obs_regions = np.zeros((row_length, col_length))
for cluster_i in range(len(clusters_vertices)):
    if cluster_i > 0:
        cluster_i_vertices = np.array(clusters_vertices[cluster_i])
        cluster_i_poly = Polygon(clusters_vertices_poly[cluster_i])
        left = cluster_i_vertices[:,1].min()
        right = cluster_i_vertices[:,1].max()
        top = cluster_i_vertices[:,0].min()
        bottom = cluster_i_vertices[:,0].max()
        for row in range(top, bottom + 1):
            for col in range(left, right + 1):
                temp_grid = Point(row, col)
                if temp_grid.within(cluster_i_poly) == True:
                    obs_regions[row, col] = cluster_i

In [4]:
# Save to tif
driver = gdal.GetDriverByName('GTiff')
obs_regions_dataset = driver.Create(data_path + 'obs_regions.tif', col_length, row_length, 1, gdal.GDT_Int32)
obs_regions_dataset.SetGeoTransform(tif_geotrans)
obs_regions_dataset.SetProjection(tif_proj)
obs_regions_dataset.GetRasterBand(1).WriteArray(obs_regions)
# Save finish.
del obs_regions_dataset